# 0. Imports

## 0.1. Libraries and configs

In [1]:
# Typing
import typing as t

# Data manipulation
import re
import pandas as pd
import numpy as np
from inflection import underscore
# from pandas_profiling import ProfileReport

# Data visualization
# import seaborn as sns
# from matplotlib import pyplot as plt
# from plotly import express as px

# Machine learning
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.decomposition import PCA
# from umap.umap_ import UMAP
# from sklearn.manifold import TSNE
# from sklearn.cluster import KMeans
# from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
# from sklearn.metrics import silhouette_score

# other
# import joblib
from sys import path
path.append("../")
from src.utils import set_jupyter_settings
from src.__logs__.logging import logger
set_jupyter_settings()


[10/27/23 09:19:43] INFO     jupyter configurations setted                                              ]8;id=271493;file:///home/sheep/repos/insiders-clustering/experiments/../src/utils.py\utils.py]8;;\:]8;id=536110;file:///home/sheep/repos/insiders-clustering/experiments/../src/utils.py#57\57]8;;\

## 0.2. Data acquisition
- Data source: [Kaggle](https://www.kaggle.com/vik2012kvs/high-value-customers-identification)
- Input description: E-commerce purchase history from November 2016 to December 2017

In [2]:
raw_df: pd.DataFrame = pd.read_csv("../data/raw/Ecommerce.csv", encoding="ISO-8859-1")
raw_df.drop(raw_df.filter(regex="Unnamed"), axis=1, inplace=True)
raw_df.sample(5)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
387967,570410,22396,MAGNETS PACK OF 4 RETRO PHOTO,1,8-Oct-17,0.39,16776.0,United Kingdom
510721,579467,23560,SET OF 6 RIBBONS COUNTRY STYLE,6,27-Nov-17,2.89,15755.0,United Kingdom
416072,572550,23567,EGG CUP HENRIETTA HEN PINK,1,22-Oct-17,2.46,NaN,United Kingdom
437992,574301,20749,ASSORTED COLOUR MINI CASES,4,1-Nov-17,7.95,12544.0,Spain
166468,550921,21977,PACK OF 60 PINK PAISLEY CAKE CASES,24,19-Apr-17,0.55,14770.0,United Kingdom


# 1. Data cleaning and description

## Data dictionary

Column | Explanation
------- | -------
**InvoiceNo** | Purchase identifier
**StockCode** | Product identifier
**Description** | Product description
**Quantity** | Quantity purchased
**InvoiceDate** | Purchase date
**UnitPrice** | Product price
**CustomerID** | Customer ID who purchased
**Country** | Where the purchase was made

## 1.1. Rename columns

In [3]:
raw_df.columns: t.List[str] = map(underscore, raw_df.columns)
print(raw_df.columns)


Index(['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')


## 1.2. Data dimensions

In [4]:
print("Rows count:\t %d\nColumns count:\t %d" % raw_df.shape)


Rows count:	 541909
Columns count:	 8


## 1.3. Duplicate data

In [5]:
raw_df.loc[raw_df.duplicated(), :]
# Duplicate rows just add noise to the data,
# so we can directly drop them because they will be useless
# for the analysis
raw_df.drop_duplicates(inplace=True)


## 1.4. Data types

In [6]:
print(raw_df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 536640 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   invoice_no    536640 non-null  object 
 1   stock_code    536640 non-null  object 
 2   description   535186 non-null  object 
 3   quantity      536640 non-null  int64  
 4   invoice_date  536640 non-null  object 
 5   unit_price    536640 non-null  float64
 6   customer_id   401603 non-null  float64
 7   country       536640 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 36.8+ MB
None


In [7]:
raw_df.loc[:, "invoice_date"] = pd.to_datetime(raw_df.loc[:, "invoice_date"], format="%d-%b-%y")
for column in ["invoice_no", "stock_code", "country"]:
    raw_df.loc[:, column] = raw_df.loc[:, column].astype("category")
raw_df.sample(5)


,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
232499,557322,22617,BAKING SET SPACEBOY DESIGN,6,2017-06-18 00:00:00,4.95,17444.0,Canada
405657,571716,21822,GLITTER CHRISTMAS TREE WITH BELLS,2,2017-10-16 00:00:00,1.63,NaN,United Kingdom
333214,566189,20727,LUNCH BAG BLACK SKULL.,3,2017-09-07 00:00:00,1.65,17800.0,United Kingdom
170442,551293,22355,CHARLOTTE BAG SUKI DESIGN,50,2017-04-25 00:00:00,0.72,14298.0,United Kingdom
13044,537415,22956,36 FOIL HEART CAKE CASES,6,2016-12-04 00:00:00,2.10,16654.0,United Kingdom


## 1.5. Missing data

In [8]:
na_summary_df = pd.DataFrame()
na_summary_df["absolute"] = raw_df.isna().sum().sort_values(ascending=False)
na_summary_df["percentage"] = na_summary_df["absolute"] / raw_df.shape[0]
na_summary_df


,absolute,percentage
customer_id,135037,0.251634
description,1454,0.002709
invoice_no,0,0.000000
stock_code,0,0.000000
quantity,0,0.000000
invoice_date,0,0.000000
unit_price,0,0.000000
country,0,0.000000


In [9]:
# At first, I'll just drop the rows with NA.
# TODO: Later on, try to deal with them more cleverly
raw_df.dropna(inplace=True)
print(raw_df.isna().sum())


invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
country         0
dtype: int64


## 1.6. Outliers

In [10]:
# TODO


## 1.7. Imbalance

In [11]:
# TODO


## 1.8. Descriptive statistics

In [23]:
num_attr_df = raw_df.select_dtypes(include=np.number)
cat_attr_df = raw_df.select_dtypes(exclude=[np.number, np.datetime64])


### 1.8.1. Numerical attributes
- Central tendency and dispersion metrics

In [28]:
num_attr_metrics_df = num_attr_df.describe().T
num_attr_metrics_df["range"] = num_attr_metrics_df["max"] - num_attr_metrics_df["min"]
num_attr_metrics_df["skew"] = num_attr_df.skew()
num_attr_metrics_df["kurtosis"] = num_attr_df.kurtosis()
num_attr_metrics_df


,count,mean,std,min,25%,50%,75%,max,range,skew,kurtosis
quantity,401603.0,12.183298,250.283348,-80995.0,2.00,5.00,12.00,80995.0,161990.0,0.179956,93140.042875
unit_price,401603.0,3.474067,69.764122,0.0,1.25,1.95,3.75,38970.0,38970.0,449.315686,243760.343381
customer_id,401603.0,15281.162093,1714.008032,12346.0,13939.00,15145.00,16784.00,18287.0,5941.0,0.034213,-1.179865


###      1.8.2. Categorical attributes

In [29]:
cat_attr_df.describe().T


,count,unique,top,freq
invoice_no,401603,22190,576339,542
stock_code,401603,3684,85123A,2065
description,401603,3896,WHITE HANGING HEART T-LIGHT HOLDER,2058
invoice_date,401603,305,2017-12-03 00:00:00,3359
country,401603,37,United Kingdom,356727
